# SECTIE 1: Setup en Configuratie
# ===============================
# Import benodigde bibliotheken en stel visualisatie-instellingen in


In [1]:

import sys
import os
import json
import traceback
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from scipy import stats
import MetaTrader5 as mt5
from sklearn.model_selection import TimeSeriesSplit
from IPython.display import display, HTML, clear_output
import warnings

# Onderdruk waarschuwingen
warnings.filterwarnings('ignore')

# Visualisatie-instellingen
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16, 9)
plt.rcParams['lines.linewidth'] = 1.5
sns.set_style("whitegrid")
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Project paden configureren
notebook_dir = os.path.abspath('')
project_path = os.path.abspath(os.path.join(notebook_dir, '..'))
print(f"Project pad: {project_path}")
sys.path.append(project_path)

# Hulpfunctie voor foutafhandeling
def display_error(e):
    """Gedetailleerde foutinformatie weergeven"""
    print(f"\n❌ FOUT: {type(e).__name__}: {e}")
    traceback.print_exc()
    print("\nControleer de foutdetails hierboven en los het probleem op voordat u doorgaat.")

# Importeren van TurtleTrader modules
try:
    # Importeer core modules
    from modules.mt5_connector import MT5Connector
    from modules.strategy import TurtleStrategy
    from modules.risk_manager import RiskManager
    from modules.backtester import Backtester
    from utils.logger import Logger
    from utils.visualizer import Visualizer
    print("✅ TurtleTrader modules succesvol geïmporteerd")
except ImportError as e:
    display_error(e)
    raise ImportError("Kon TurtleTrader modules niet importeren. Controleer het project pad.")

print("Setup voltooid. Klaar om te starten met validatie.")

Project pad: C:\Users\basti\PycharmProjects\TurtleTrader
✅ TurtleTrader modules succesvol geïmporteerd
Setup voltooid. Klaar om te starten met validatie.


# SECTIE 2: Configuratie Laden
# ===========================
# Laad de TurtleTrader configuratie uit het JSON-bestand

In [2]:


def load_config(config_path):
    """
    Laad configuratie uit JSON bestand met foutafhandeling

    Parameters:
    -----------
    config_path : str
        Pad naar het configuratiebestand

    Returns:
    --------
    dict
        Geladen configuratie als dictionary
    """
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        return config
    except FileNotFoundError:
        raise FileNotFoundError(f"Configuratiebestand niet gevonden op {config_path}. Controleer het pad.")
    except json.JSONDecodeError:
        raise ValueError(f"Ongeldige JSON in configuratiebestand {config_path}. Controleer het bestandsformaat.")

try:
    # Configuratie laden
    config_path = os.path.join(project_path, 'config', 'turtle_settings.json')
    print(f"Configuratie zoeken op: {config_path}")

    # Controleer of het bestand bestaat
    if not os.path.exists(config_path):
        print(f"⚠️ Configuratiebestand niet gevonden op de verwachte locatie.")
        print(f"Laten we de mapstructuur controleren:")

        # Mapstructuur weergeven voor debugging
        project_contents = os.listdir(project_path)
        print(f"Inhoud van {project_path}:")
        for item in sorted(project_contents):
            if os.path.isdir(os.path.join(project_path, item)):
                print(f"  📁 {item}/")
            else:
                print(f"  📄 {item}")

    # Configuratie laden
    config = load_config(config_path)

    # Configuratie weergeven
    print("\nTurtleTrader Configuratie:")
    print(f"Broker: {config['mt5']['server']}")
    print(f"Login: {config['mt5']['login']}")
    print(f"MT5 pad: {config['mt5']['mt5_pathway']}")
    print(f"Symbolen: {config['mt5']['symbols']}")
    print(f"Timeframe: {config['mt5']['timeframe']}")
    print(f"Risico per trade: {config['mt5']['risk_per_trade'] * 100}%")

    if 'symbol_mapping' in config['mt5']:
        print(f"Symbool mapping: {config['mt5']['symbol_mapping']}")

    # Voor makkelijke referentie in het notebook
    mt5_config = config['mt5']

    # Definieer timeframe mapping voor later gebruik
    timeframe_dict = {
        "M1": mt5.TIMEFRAME_M1,
        "M5": mt5.TIMEFRAME_M5,
        "M15": mt5.TIMEFRAME_M15,
        "M30": mt5.TIMEFRAME_M30,
        "H1": mt5.TIMEFRAME_H1,
        "H4": mt5.TIMEFRAME_H4,
        "D1": mt5.TIMEFRAME_D1,
        "W1": mt5.TIMEFRAME_W1,
        "MN1": mt5.TIMEFRAME_MN1
    }

    # Gebruik de timeframe uit de configuratie
    if 'timeframe' in mt5_config and mt5_config['timeframe'] in timeframe_dict:
        mt5_timeframe = timeframe_dict[mt5_config['timeframe']]
        print(f"Gebruik timeframe: {mt5_config['timeframe']} (MT5 constant: {mt5_timeframe})")
    else:
        mt5_timeframe = mt5.TIMEFRAME_D1
        print(f"Gebruik standaard timeframe: D1 (MT5 constant: {mt5_timeframe})")

    print("\nConfiguratie succesvol geladen.")
except Exception as e:
    display_error(e)
    raise Exception("Kon configuratie niet laden. Zie details hierboven.")
# Beschikbare symbolen ophalen en printen
try:
    if not mt5.initialize():
        raise RuntimeError("MT5 initialisatie mislukt.")

    print("\nBeschikbare symbolen bij broker:")
    symbols = mt5.symbols_get()
    for symbol in symbols:
        print(symbol.name)

    mt5.shutdown()
except Exception as e:
    print(f"Fout bij ophalen van symbolen: {e}")


Configuratie zoeken op: C:\Users\basti\PycharmProjects\TurtleTrader\config\turtle_settings.json

TurtleTrader Configuratie:
Broker: FTMO-Demo2
Login: 1520533067
MT5 pad: C:\Program Files\FTMO MetaTrader 5\terminal64.exe
Symbolen: ['EURUSD', 'XAUUSD', 'US30']
Timeframe: D1
Risico per trade: 1.0%
Symbool mapping: {'US30': 'US30.cash'}
Gebruik timeframe: D1 (MT5 constant: 16408)

Configuratie succesvol geladen.

Beschikbare symbolen bij broker:
EURUSD
GBPUSD
USDCHF
USDJPY
USDCAD
AUDUSD
AUDNZD
AUDCAD
AUDCHF
AUDJPY
CHFJPY
EURGBP
EURAUD
EURCHF
EURJPY
EURNZD
EURCAD
GBPCHF
GBPJPY
CADCHF
CADJPY
GBPAUD
GBPCAD
GBPNZD
NZDCAD
NZDCHF
NZDJPY
NZDUSD
GER40.cash
UK100.cash
US100.cash
US30.cash
US500.cash
AUS200.cash
EU50.cash
FRA40.cash
HK50.cash
JP225.cash
SPN35.cash
US2000.cash
N25.cash
UKOIL.cash
USOIL.cash
XAUUSD
XAUAUD
XAUEUR
XAGAUD
XAGEUR
XAGUSD
XPDUSD
XPTUSD
ADAUSD
BTCUSD
DASHUSD
DOGEUSD
DOTUSD
ETHUSD
LTCUSD
NEOUSD
XMRUSD
XRPUSD
EURCZK
EURHUF
EURNOK
EURPLN
USDCZK
USDHKD
USDHUF
USDILS
USDMXN
USDNO

# SECTIE 3: MT5 Verbinding Testen
# ==============================
# Test de verbinding met MetaTrader 5 en verkrijg marktgegevens

In [3]:


def test_mt5_connection(config):
    """
    Test de verbinding met MetaTrader 5 en geef informatie weer over het account en de beschikbare symbolen

    Parameters:
    -----------
    config : dict
        MT5 configuratie-instellingen

    Returns:
    --------
    MT5Connector of None
        Een geïnitialiseerde connector bij succes, None bij mislukking
    """
    print("\n" + "="*50)
    print("MT5 VERBINDINGSTEST")
    print("="*50 + "\n")

    try:
        # Initialiseer connector
        connector = MT5Connector(config)
        start_time = datetime.now()
        connected = connector.connect()
        connection_time = (datetime.now() - start_time).total_seconds()

        if connected:
            print(f"✅ MT5 verbinding succesvol! (Verbindingstijd: {connection_time:.2f} seconden)")

            # Account informatie
            account_info = connector.get_account_info()
            print(f"\nAccount Informatie:")
            print(f"  Server: {mt5.account_info().server}")
            print(f"  Balance: ${account_info.get('balance', 'Onbekend'):,.2f}")
            print(f"  Equity: ${account_info.get('equity', 'Onbekend'):,.2f}")
            print(f"  Margin: ${account_info.get('margin', 'Onbekend'):,.2f}")
            print(f"  Free Margin: ${account_info.get('free_margin', 'Onbekend'):,.2f}")
            print(f"  Profit: ${account_info.get('profit', 'Onbekend'):,.2f}")

            # Test symbolen
            print("\nSymbool Status:")
            symbol_table = []

            for symbol in config['symbols']:
                try:
                    # Pas symbool mapping toe indien nodig
                    mapped_symbol = symbol
                    if 'symbol_mapping' in config and symbol in config['symbol_mapping']:
                        mapped_symbol = config['symbol_mapping'][symbol]

                    # Haal symbool info op
                    symbol_info = mt5.symbol_info(mapped_symbol)
                    if symbol_info is not None:
                        print(f"  ✅ {symbol} (als {mapped_symbol}): Beschikbaar")
                        tick = connector.get_symbol_tick(symbol)
                        if tick is not None:
                            spread = (tick.ask - tick.bid) / tick.bid * 100 if tick.bid > 0 else 0
                            print(f"     Bid: {tick.bid:.5f}, Ask: {tick.ask:.5f}, Spread: {spread:.4f}%")

                            # Verzamel symbool informatie voor tabel
                            symbol_table.append({
                                'Symbool': symbol,
                                'Mapping': mapped_symbol if mapped_symbol != symbol else '-',
                                'Bid': tick.bid,
                                'Ask': tick.ask,
                                'Spread (%)': spread,
                                'Volume': tick.volume,
                                'Tijd': datetime.fromtimestamp(tick.time)
                            })
                    else:
                        print(f"  ❌ {symbol} (als {mapped_symbol}): Niet beschikbaar")
                except Exception as e:
                    print(f"  ❌ {symbol}: Fout bij ophalen info: {e}")

            # Toon samenvattingstabel indien er symbolen zijn
            if symbol_table:
                print("\nSymbool Overzicht:")
                symbol_df = pd.DataFrame(symbol_table)
                display(symbol_df)

            return connector
        else:
            print(f"❌ Kon geen verbinding maken met MT5.")
            print(f"Controleer of MT5 is geïnstalleerd op het opgegeven pad en dat de inloggegevens correct zijn.")
            return None
    except Exception as e:
        display_error(e)
        return None

# Voer verbindingstest uit
print("Start MT5 verbindingstest...")
connector = test_mt5_connection(mt5_config)

if connector is not None:
    print("\nVerbinding met MT5 succesvol tot stand gebracht. Klaar om verder te gaan met data-analyse.")
else:
    print("\n⚠️ MT5 verbinding niet tot stand gebracht. Controleer bovenstaande foutmeldingen.")
    print("Opmerking: Sommige validaties kunnen nog steeds worden uitgevoerd zonder live verbinding.")

Start MT5 verbindingstest...

MT5 VERBINDINGSTEST

Symbool mapping toegepast: US30 -> US30.cash
Verbonden met MT5: FTMO-Demo2
✅ MT5 verbinding succesvol! (Verbindingstijd: 0.00 seconden)

Account Informatie:
  Server: FTMO-Demo2
  Balance: $79,997.56
  Equity: $79,997.56
  Margin: $0.00
  Free Margin: $79,997.56
  Profit: $0.00

Symbool Status:
  ✅ EURUSD (als EURUSD): Beschikbaar
     Bid: 1.03754, Ask: 1.03763, Spread: 0.0087%
  ✅ XAUUSD (als XAUUSD): Beschikbaar
     Bid: 2858.42000, Ask: 2859.10000, Spread: 0.0238%
  ✅ US30 (als US30.cash): Beschikbaar
     Bid: 43772.83000, Ask: 43775.16000, Spread: 0.0053%

Symbool Overzicht:


,Symbool,Mapping,Bid,Ask,Spread (%),Volume,Tijd
0,EURUSD,-,1.03754,1.03763,0.008674,0,2025-03-01 00:54:59
1,XAUUSD,-,2858.42000,2859.10000,0.023789,0,2025-03-01 00:49:54
2,US30,US30.cash,43772.83000,43775.16000,0.005323,0,2025-03-01 00:49:48



Verbinding met MT5 succesvol tot stand gebracht. Klaar om verder te gaan met data-analyse.


# SECTIE 4: Historische Data Ophalen en Valideren
# ==============================================
# Haal historische data op voor elk symbool en voer uitgebreide validatie uit

In [ ]:
# SECTIE 4: Intelligente Data Validatie voor Alle Symbolen
# ========================================================

def fetch_and_validate_data(connector, symbol, timeframe, bars_count, check_outliers=True):
    """
    Haal historische data op en voer uitgebreide validatie uit

    Parameters:
    -----------
    connector : MT5Connector
        Verbinding met MetaTrader 5
    symbol : str
        Het te analyseren symbool
    timeframe : int
        MT5 timeframe constante
    bars_count : int
        Aantal candles om op te halen
    check_outliers : bool, optional
        Of outliers gedetecteerd moeten worden

    Returns:
    --------
    pandas.DataFrame of None
        Gevalideerde historische data of None bij fouten
    """
    print("\n" + "="*50)
    print(f"DATA VALIDATIE: {symbol}")
    print("="*50 + "\n")

    if not connector or not connector.connected:
        print("❌ Geen verbinding met MT5. Kan geen data ophalen.")
        return None

    try:
        # Data ophalen met timing
        start_time = datetime.now()
        df = connector.get_historical_data(symbol, timeframe, bars_count)
        fetch_time = (datetime.now() - start_time).total_seconds()

        if df is None or df.empty:
            print(f"❌ Geen data opgehaald voor {symbol}")
            return None

        print(f"✅ Data succesvol opgehaald! ({len(df)} candles, tijd: {fetch_time:.2f} seconden)")
        print(f"Periode: {df['time'].min()} tot {df['time'].max()}")

        # Basis data informatie
        print(f"\nDataframe info: {df.shape[0]} rijen × {df.shape[1]} kolommen")
        print(f"Kolommen: {', '.join(df.columns.tolist())}")
        print("\nDataframe head (eerste 5 rijen):")
        display(df.head())

        # ===== DATA INTEGRITEITSCONTROLES =====
        print("\n" + "-"*30)
        print("DATA INTEGRITEITSCONTROLE")
        print("-"*30)

        validation_issues = []

        # 1. Controleer op NaN-waarden
        nan_counts = df.isna().sum()
        if nan_counts.sum() > 0:
            print("⚠️ NaN-waarden gevonden:")
            nan_cols = nan_counts[nan_counts > 0]
            for col, count in nan_cols.items():
                pct = count / len(df) * 100
                print(f"  - {col}: {count} waarden ({pct:.2f}%)")
                validation_issues.append(f"NaN in {col}: {count} waarden")
        else:
            print("✅ Geen NaN-waarden gevonden")

        # 2. Controleer op duplicaten
        if df['time'].duplicated().any():
            duplicate_count = df['time'].duplicated().sum()
            dup_pct = duplicate_count / len(df) * 100
            print(f"⚠️ {duplicate_count} dubbele tijdstempels gevonden ({dup_pct:.2f}%)")
            validation_issues.append(f"Duplicaten: {duplicate_count} tijdstempels")

            # Toon enkele voorbeelden van duplicaten
            dup_times = df[df['time'].duplicated(keep=False)]['time'].sort_values().unique()
            if len(dup_times) > 0:
                print("  Voorbeelden van dubbele tijdstempels:")
                for i, time in enumerate(dup_times[:3]):
                    print(f"  - {time}")
                if len(dup_times) > 3:
                    print(f"    ... en {len(dup_times) - 3} meer")
        else:
            print("✅ Geen dubbele tijdstempels gevonden")

        # 3. Controleer op gaten in de tijdreeks
        df = df.sort_values('time').reset_index(drop=True)
        df['time_diff'] = df['time'].diff()

        if timeframe == mt5.TIMEFRAME_D1:
            # Voor dagelijkse data, controleer op ontbrekende handelsdagen (zonder weekenden)
            df['weekday'] = df['time'].dt.dayofweek
            gaps = df[(df['time_diff'] > timedelta(days=1)) & (df['weekday'] != 0)]

            if len(gaps) > 0:
                gap_pct = len(gaps) / len(df) * 100
                print(f"⚠️ {len(gaps)} gaten in de tijdreeks gevonden ({gap_pct:.2f}%, exclusief weekenden)")
                validation_issues.append(f"Tijdreeksgaten: {len(gaps)} gaten")

                print("  Voorbeelden van gaten (exclusief weekenden):")
                sample_gaps = gaps.head(min(3, len(gaps)))
                for _, row in sample_gaps.iterrows():
                    print(f"  - Gap van {row['time_diff'].days} dagen tussen {row['time'] - row['time_diff']} en {row['time']}")
            else:
                print("✅ Geen onverwachte gaten in de tijdreeks")
        else:
            # Voor andere timeframes een eenvoudigere controle
            expected_diff = {
                mt5.TIMEFRAME_M1: timedelta(minutes=1),
                mt5.TIMEFRAME_M5: timedelta(minutes=5),
                mt5.TIMEFRAME_M15: timedelta(minutes=15),
                mt5.TIMEFRAME_M30: timedelta(minutes=30),
                mt5.TIMEFRAME_H1: timedelta(hours=1),
                mt5.TIMEFRAME_H4: timedelta(hours=4),
                mt5.TIMEFRAME_W1: timedelta(days=7),
                mt5.TIMEFRAME_MN1: timedelta(days=28)  # Benadering
            }

            if timeframe in expected_diff:
                expected = expected_diff[timeframe]
                # We slaan weekenden etc. over, dus check alleen items die >2x de expected diff zijn
                significant_gaps = df[df['time_diff'] > expected * 2].copy()

                if len(significant_gaps) > 0:
                    gap_pct = len(significant_gaps) / len(df) * 100
                    print(f"⚠️ {len(significant_gaps)} significante gaten in de tijdreeks gevonden ({gap_pct:.2f}%)")
                    validation_issues.append(f"Tijdreeksgaten: {len(significant_gaps)} gaten")

                    print("  Voorbeelden van significante gaten:")
                    sample_gaps = significant_gaps.head(min(3, len(significant_gaps)))
                    for _, row in sample_gaps.iterrows():
                        print(f"  - Gap van {row['time_diff']} tussen {row['time'] - row['time_diff']} en {row['time']}")
                else:
                    print("✅ Geen significante gaten in de tijdreeks")
            else:
                print("ℹ️ Gatencontrole niet beschikbaar voor deze timeframe")

        # 4. Controleer op outliers indien gevraagd
        if check_outliers:
            print("\n" + "-"*30)
            print("OUTLIER DETECTIE")
            print("-"*30)

            # Z-score methode
            z_threshold = 3.0
            numeric_cols = ['open', 'high', 'low', 'close', 'tick_volume']
            outlier_counts = {}

            for col in numeric_cols:
                if col in df.columns:
                    z_scores = np.abs(stats.zscore(df[col].dropna()))
                    outliers = np.where(z_scores > z_threshold)[0]

                    if len(outliers) > 0:
                        outlier_pct = len(outliers) / len(df) * 100
                        print(f"⚠️ {col}: {len(outliers)} outliers gevonden ({outlier_pct:.2f}%)")
                        outlier_counts[col] = len(outliers)
                        validation_issues.append(f"Outliers in {col}: {len(outliers)} waarden")

                        # Toon enkele voorbeelden
                        if len(outliers) > 0:
                            outlier_indices = outliers[:min(3, len(outliers))]
                            print("  Voorbeelden van outliers:")
                            for idx in outlier_indices:
                                if idx < len(df):
                                    z_val = z_scores[idx]
                                    print(f"  - {df.iloc[idx]['time']}: {col}={df.iloc[idx][col]:.5f} (z-score: {z_val:.2f})")
                    else:
                        print(f"✅ {col}: Geen outliers gevonden")

            # Plot outliers als er zijn gevonden (optioneel)
            if any(outlier_counts.values()) and False:  # Set to True to enable plots
                plt.figure(figsize=(14, 8))
                plt.subplot(2, 1, 1)
                plt.plot(df['time'], df['close'], '-b', alpha=0.7, label='Sluitingskoers')

                # Highlight potential outliers in price
                if 'close' in outlier_counts and outlier_counts['close'] > 0:
                    z_scores = np.abs(stats.zscore(df['close'].dropna()))
                    outlier_mask = z_scores > z_threshold
                    plt.scatter(df.loc[outlier_mask, 'time'], df.loc[outlier_mask, 'close'],
                                color='red', s=50, label='Outliers')

                plt.title(f'Prijsdata met Outliers - {symbol}')
                plt.ylabel('Prijs')
                plt.legend()
                plt.grid(True, alpha=0.3)

                # Plot volume
                plt.subplot(2, 1, 2)
                plt.bar(df['time'], df['tick_volume'], color='blue', alpha=0.5, label='Volume')

                # Highlight potential outliers in volume
                if 'tick_volume' in outlier_counts and outlier_counts['tick_volume'] > 0:
                    z_scores = np.abs(stats.zscore(df['tick_volume'].dropna()))
                    outlier_mask = z_scores > z_threshold
                    plt.scatter(df.loc[outlier_mask, 'time'], df.loc[outlier_mask, 'tick_volume'],
                                color='red', s=50, label='Volume Outliers')

                plt.title('Volume met Outliers')
                plt.ylabel('Volume')
                plt.legend()
                plt.tight_layout()
                plt.show()

        # 5. Prijssprong detectie (extreme bewegingen)
        print("\n" + "-"*30)
        print("PRIJSSPRONG DETECTIE")
        print("-"*30)

        df['pct_change'] = df['close'].pct_change() * 100
        extreme_moves = df[abs(df['pct_change']) > 2.0]  # Bewegingen >2% als voorbeeld

        if len(extreme_moves) > 0:
            move_pct = len(extreme_moves) / len(df) * 100
            print(f"ℹ️ {len(extreme_moves)} significante prijsbewegingen (>2%) gevonden ({move_pct:.2f}%)")

            print("  Top 3 grootste bewegingen:")
            top_moves = extreme_moves.sort_values('pct_change', key=abs, ascending=False).head(3)
            for _, row in top_moves.iterrows():
                print(f"  - {row['time']}: {row['pct_change']:.2f}% ({row['close']:.5f})")
        else:
            print("ℹ️ Geen extreme prijsbewegingen (>2%) gevonden")

        # 6. Beschrijvende statistieken
        print("\n" + "-"*30)
        print("BESCHRIJVENDE STATISTIEKEN")
        print("-"*30)

        stats_df = df[['open', 'high', 'low', 'close', 'tick_volume']].describe()
        display(stats_df)

        # 7. Validatie samenvatting
        print("\n" + "-"*30)
        print("VALIDATIE SAMENVATTING")
        print("-"*30)

        if validation_issues:
            print(f"⚠️ {len(validation_issues)} potentiële validatie-problemen gevonden:")
            for issue in validation_issues:
                print(f"  - {issue}")
            print("\nOverweeg of deze problemen de strategie-evaluatie kunnen beïnvloeden.")
        else:
            print("✅ Geen validatie-problemen gevonden. Data lijkt van hoge kwaliteit.")

        return df
    except Exception as e:
        display_error(e)
        return None

# Historische gegevens ophalen en valideren voor alle symbolen
if connector:
    # Aantal candles om op te halen (2 jaar dagelijkse data voor backtest)
    days_back = 365 * 2
    bars_count = days_back
    if mt5_config['timeframe'] == 'H1':
        bars_count = days_back * 24
    elif mt5_config['timeframe'] == 'H4':
        bars_count = days_back * 6

    print("\n" + "="*50)
    print("INTELLIGENTE DATA VALIDATIE VOOR ALLE SYMBOLEN")
    print("="*50 + "\n")

    # Gegevens voor alle symbolen ophalen en valideren
    symbol_data = {}
    validation_summaries = []

    # Loop door alle symbolen
    for symbol in mt5_config['symbols']:
        print(f"\n\n{'#'*70}")
        print(f"### VALIDATIE VOOR {symbol} ###")
        print(f"{'#'*70}\n")

        df = fetch_and_validate_data(connector, symbol, mt5_timeframe, bars_count)

        if df is not None:
            symbol_data[symbol] = df

            # Verzamel validatie-metrics voor vergelijking
            metrics = {
                'symbol': symbol,
                'candles': len(df),
                'period_start': df['time'].min(),
                'period_end': df['time'].max(),
                'avg_price': df['close'].mean(),
                'volatility': df['close'].pct_change().std() * 100,  # Dagelijkse volatiliteit in %
                'gaps': len(df[(df['time_diff'] > timedelta(days=1)) & (df['weekday'] != 0)]) if 'weekday' in df.columns else 'N/A',
                'outliers_close': len(df[abs(stats.zscore(df['close'])) > 3]) if len(df) > 30 else 'N/A',
                'big_moves': len(df[abs(df['pct_change']) > 2.0]) if 'pct_change' in df.columns else 'N/A',
                'avg_volume': df['tick_volume'].mean()
            }
            validation_summaries.append(metrics)

            print(f"\n✅ {symbol} data succesvol opgehaald en gevalideerd.")
        else:
            print(f"\n❌ Kon geen data valideren voor {symbol}.")

    # Als we minstens één symbool hebben verwerkt, toon dan een vergelijkend overzicht
    if validation_summaries:
        print("\n" + "="*80)
        print("VERGELIJKEND OVERZICHT VAN ALLE SYMBOLEN")
        print("="*80 + "\n")

        # Converteer naar DataFrame voor mooie weergave
        summary_df = pd.DataFrame(validation_summaries).set_index('symbol')

        # Bereken extra vergelijkingsmetrieken
        if len(summary_df) > 1:
            # Normaliseer volatiliteit (hoger = meer volatiel)
            max_vol = summary_df['volatility'].max()
            if max_vol > 0:
                summary_df['rel_volatility'] = summary_df['volatility'] / max_vol

            # Bereken een 'trading potential score' (hoger = beter)
            summary_df['trading_score'] = 0.0
            for idx, row in summary_df.iterrows():
                # Base score op volatiliteit en big moves, indien beschikbaar
                vol_score = min(row['volatility'] * 10, 100) if isinstance(row['volatility'], (int, float)) else 50
                big_moves_score = min(row['big_moves'] * 20, 100) if isinstance(row['big_moves'], (int, float)) and row['big_moves'] != 'N/A' else 50
                summary_df.at[idx, 'trading_score'] = (vol_score + big_moves_score) / 2

        # Print het vergelijkend overzicht
        display(summary_df)

        # Visualiseer prijsbewegingen van alle symbolen voor vergelijking
        if len(symbol_data) > 1:
            plt.figure(figsize=(14, 7))

            # Normaliseer prijzen voor betere vergelijking
            for symbol, data in symbol_data.items():
                # Bereken genormaliseerde prijs (eerste waarde = 100)
                normalized = data['close'] / data['close'].iloc[0] * 100
                plt.plot(data['time'], normalized, label=symbol)

            plt.title('Vergelijking van Genormaliseerde Prijzen (Startwaarde = 100)')
            plt.xlabel('Datum')
            plt.ylabel('Genormaliseerde Prijs')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()

            # Volatiliteit vergelijking
            plt.figure(figsize=(10, 6))
            volatilities = [df['close'].pct_change().std() * 100 for df in symbol_data.values()]
            plt.bar(symbol_data.keys(), volatilities)
            plt.title('Volatiliteit Vergelijking (Dagelijkse standaarddeviatie in %)')
            plt.ylabel('Volatiliteit (%)')
            plt.grid(True, alpha=0.3, axis='y')
            plt.show()

        # Toon aanbeveling op basis van analyse
        if len(summary_df) > 1 and 'trading_score' in summary_df.columns:
            best_symbol = summary_df['trading_score'].idxmax()
            print(f"\n🔍 Analyse: Op basis van de historische data lijkt {best_symbol} het meest geschikt voor de TurtleTrader strategie")
            print(f"   vanwege de combinatie van volatiliteit en significante prijsbewegingen.")

    # Aantal symbolen waarvoor we succesvolle gegevens hebben verzameld
    success_count = len(symbol_data)
    total_count = len(mt5_config['symbols'])

    print(f"\nSuccesvol gegevens verzameld voor {success_count}/{total_count} symbolen.")
else:
    print("Kon geen verbinding maken met MT5. Kan geen historische data ophalen.")


INTELLIGENTE DATA VALIDATIE VOOR ALLE SYMBOLEN



######################################################################
### VALIDATIE VOOR EURUSD ###
######################################################################


DATA VALIDATIE: EURUSD

✅ Data succesvol opgehaald! (730 candles, tijd: 0.00 seconden)
Periode: 2022-05-06 00:00:00 tot 2025-02-27 00:00:00

Dataframe info: 730 rijen × 8 kolommen
Kolommen: time, open, high, low, close, tick_volume, spread, real_volume

Dataframe head (eerste 5 rijen):


,time,open,high,low,close,tick_volume,spread,real_volume
0,2022-05-06,1.05389,1.05988,1.04828,1.05440,90213,2,0
1,2022-05-09,1.05410,1.05924,1.04951,1.05603,73029,2,0
2,2022-05-10,1.05574,1.05853,1.05254,1.05286,74069,2,0
3,2022-05-11,1.05285,1.05770,1.05019,1.05129,76697,2,0
4,2022-05-12,1.05134,1.05293,1.03541,1.03817,81792,2,0



------------------------------
DATA INTEGRITEITSCONTROLE
------------------------------
✅ Geen NaN-waarden gevonden
✅ Geen dubbele tijdstempels gevonden
⚠️ 5 gaten in de tijdreeks gevonden (0.68%, exclusief weekenden)
  Voorbeelden van gaten (exclusief weekenden):
  - Gap van 4 dagen tussen 2022-12-23 00:00:00 en 2022-12-27 00:00:00
  - Gap van 4 dagen tussen 2023-12-22 00:00:00 en 2023-12-26 00:00:00
  - Gap van 4 dagen tussen 2023-12-29 00:00:00 en 2024-01-02 00:00:00

------------------------------
OUTLIER DETECTIE
------------------------------
⚠️ open: 3 outliers gevonden (0.41%)
  Voorbeelden van outliers:
  - 2022-09-26 00:00:00: open=0.96695 (z-score: 3.04)
  - 2022-09-27 00:00:00: open=0.96091 (z-score: 3.22)
  - 2022-09-28 00:00:00: open=0.95893 (z-score: 3.28)
⚠️ high: 3 outliers gevonden (0.41%)
  Voorbeelden van outliers:
  - 2022-09-26 00:00:00: high=0.97098 (z-score: 3.12)
  - 2022-09-27 00:00:00: high=0.96707 (z-score: 3.25)
  - 2022-10-12 00:00:00: high=0.97348 (z-sco